# Creador de bbdd Retardadas

In [1]:
import pandas as pd
import os

In [2]:
# Cargar datos de jugadores y equipos
players_df = pd.read_csv('C:/Users/rsier/Desktop/MASTERS/LaSalle - MUDS/TFM/Data/Def_Data/Clustering_Data/Players_noGK_data_clustered.csv')
gk_df = pd.read_csv('C:/Users/rsier/Desktop/MASTERS/LaSalle - MUDS/TFM/Data/Def_Data/Clustering_Data/Players_GK_data_clustered.csv')
teams_df = pd.read_csv('C:/Users/rsier/Desktop/MASTERS/LaSalle - MUDS/TFM/Data/Def_Data/Clustering_Data/Teams_data_clustered.csv')

In [3]:
# Definir las columnas de unión
non_sufixes_cols = ['Season_End_Year', 'Squad', 'Comp', 'Player']
non_sufixes_cols_team = ['Season_End_Year', 'Squad', 'Comp']

# Añadir sufijos a las columnas restantes
players_df_suffixed = players_df.set_index(non_sufixes_cols).add_suffix('_Player').reset_index()
gk_df_suffixed = gk_df.set_index(non_sufixes_cols).add_suffix('_GK').reset_index()
teams_df_suffixed = teams_df.set_index(non_sufixes_cols_team).add_suffix('_Team').reset_index()

data_merged_players = pd.merge(players_df_suffixed, teams_df_suffixed, on=['Season_End_Year', 'Squad', 'Comp'])
data_merged_GK = pd.merge(gk_df_suffixed, teams_df_suffixed, on=['Season_End_Year', 'Squad', 'Comp'])

In [4]:
data_merged_players.head(1)

,Season_End_Year,Squad,Comp,Player,Nation_Player,Pos_Player,Age_Player,Min_Playing.Time_Generales_Player,Min_percent_Playing.Time_Generales_Player,onxG_Team.Success..xG._Generales_Player,...,Num_Players_Porteria_Team,GA90_Porteria_Team,PSxG_per_SoT_Expected_Porteria_Team,CS_percent_Porteria_Team,Att_Launched_Porteria_Team,Opp_Crosses_Porteria_Team,AvgLen_Goal_Porteria_Team,AvgLen_Passes_Porteria_Team,CK_Goals_Porteria_Team,Cluster_Team
0,2018,Alavés,La Liga,Alexis,ESP,DF,31,2184,63.9,26.4,...,2,1.32,0.25,31.6,786,657,56.7,43.4,7,5


In [5]:
# Eliminar el caso único con índice 4213 si existe en el DataFrame
if 4213 in data_merged_players.index:
    data_merged_players = data_merged_players.drop(index=4213)

# Ordenar por los criterios especificados: Primero por Player y Season_End_Year, luego por Min_Playing.Time_Generales_Player y MP_Playing.Time_Generales_Player
data_sorted = data_merged_players.sort_values(by=['Player', 'Season_End_Year', 'Min_Playing.Time_Generales_Player', 'MP_Playing.Time_Generales_Player'], ascending=[True, True, False, False])

# Agrupar por Player y Season_End_Year y seleccionar la fila con el mayor valor de Min_Playing.Time_Generales_Player y, en caso de empate, MP_Playing.Time_Generales_Player
idx = data_sorted.groupby(['Player', 'Season_End_Year'], sort=False)['Min_Playing.Time_Generales_Player'].idxmax()

# Convertir los índices a una lista unidimensional
idx = idx.values

# Seleccionar las filas correspondientes a los índices
df_grouped = data_sorted.loc[idx]

# Verificar si hay duplicados en df_grouped
duplicates = df_grouped[df_grouped.duplicated(subset=['Player', 'Season_End_Year'], keep=False)]
# print("Duplicados después de la agrupación:")
# print(duplicates)

# Eliminar duplicados si existen
df_grouped = df_grouped.drop_duplicates(subset=['Player', 'Season_End_Year'])

# Pivotar los datos
pivoted_players = df_grouped.pivot(index='Player', columns='Season_End_Year')

# Aplanar el MultiIndex en las columnas
pivoted_players.columns = [f'{var}_{year}' for var, year in pivoted_players.columns]

# Resetear el índice para convertir 'Player' de nuevo en columna
pivoted_players.reset_index(inplace=True)

# Mostrar el resultado
pivoted_players

# Definir el directorio donde se guardarán los archivos
directory = 'C:/Users/rsier/Desktop/MASTERS/LaSalle - MUDS/TFM/Data/Def_Data/Retarded_Data/'  # Cambia esto por la ruta de tu directorio

# Asegurarse de que el directorio existe
os.makedirs(directory, exist_ok=True)

# Definir las rutas completas para los archivos
players_path = os.path.join(directory, 'Players_retarded.csv')

# Exportar los DataFrames a archivos CSV
pivoted_players.to_csv(players_path, index=False)

In [6]:
# Eliminar el caso único con índice 4213 si existe en el DataFrame
# if 4213 in data_merged_GK.index:
#     data_merged_GK = data_merged_GK.drop(index=4213)

# Ordenar por los criterios especificados: Primero por Player y Season_End_Year, luego por Min_Playing.Time_Generales_Player y MP_Playing.Time_Generales_Player
data_sorted = data_merged_GK.sort_values(by=['Player', 'Season_End_Year', 'Min_Playing.Time_Generales_GK', 'MP_Playing.Time_Generales_GK'], ascending=[True, True, False, False])

# Agrupar por Player y Season_End_Year y seleccionar la fila con el mayor valor de Min_Playing.Time_Generales_Player y, en caso de empate, MP_Playing.Time_Generales_Player
idx = data_sorted.groupby(['Player', 'Season_End_Year'], sort=False)['Min_Playing.Time_Generales_GK'].idxmax()

# Convertir los índices a una lista unidimensional
idx = idx.values

# Seleccionar las filas correspondientes a los índices
df_grouped = data_sorted.loc[idx]

# Verificar si hay duplicados en df_grouped
duplicates = df_grouped[df_grouped.duplicated(subset=['Player', 'Season_End_Year'], keep=False)]
# print("Duplicados después de la agrupación:")
# print(duplicates)

# Eliminar duplicados si existen
df_grouped = df_grouped.drop_duplicates(subset=['Player', 'Season_End_Year'])

# Pivotar los datos
pivoted_gk = df_grouped.pivot(index='Player', columns='Season_End_Year')

# Aplanar el MultiIndex en las columnas
pivoted_gk.columns = [f'{var}_{year}' for var, year in pivoted_gk.columns]

# Resetear el índice para convertir 'Player' de nuevo en columna
pivoted_gk.reset_index(inplace=True)

# Mostrar el resultado
pivoted_gk

gk_path = os.path.join(directory, 'GK_retarded.csv')

# Exportar los DataFrames a archivos CSV
pivoted_gk.to_csv(gk_path, index=False)